In [1]:
import pandas as pd
import numpy as np
import math

import matplotlib.pyplot as plt

In [2]:
data = pd.read_parquet('../data/data_2022.parquet')

In [3]:
data.head()

,date,dayofweek,away_team,away_league,away_game_number,home_team,home_league,home_game_number,away_score,home_score,...,home_SF,home_HBP,home_BB,home_SO,home_p_cnt,home_putouts,home_assists,home_errors,away_P_id,home_P_id
0,20220407,Thu,SDN,NL,1,ARI,NL,1,2,4,...,0,1,7,6,6,27,9,0,darvy001,bumgm001
1,20220407,Thu,CIN,NL,1,ATL,NL,1,6,3,...,0,0,5,13,4,27,7,0,mahlt001,friem001
2,20220407,Thu,MIL,NL,1,CHN,NL,1,4,5,...,1,1,4,7,6,27,9,0,burnc002,hendk001
3,20220407,Thu,PIT,NL,1,SLN,NL,1,0,9,...,2,1,7,5,4,27,11,0,brubj001,waina001
4,20220407,Thu,NYN,NL,1,WAS,NL,1,5,1,...,0,0,2,10,6,27,10,0,megit002,corbp001


In [38]:
data_columns = ['date', 'dayofweek', 'away_team', 'away_game_number', 'away_league', 'home_team', 'home_game_number', 'home_league', \
                'home_score', 'away_score', 'park_id', 'away_P_id', 'home_P_id']
away_columns = ['away_team', 'away_game_number', 'away_AB', 'away_H', 'away_2B', 'away_3B', 'away_HR', 'away_SF', 'away_HBP', 'away_BB', 'away_SO', \
             'away_p_cnt', 'away_putouts', 'away_assists', 'away_errors']
home_columns = ['home_team', 'home_game_number', 'home_AB', 'home_H', 'home_2B', 'home_3B', 'home_HR', \
             'home_SF', 'home_HBP', 'home_BB', 'home_SO', 'home_p_cnt', 'home_putouts', 'home_assists', 'home_errors']
team_columns = ['team', 'game_number', 'AB', 'H', '2B', '3B', 'HR', 'SF', 'HBP', 'BB', 'SO', 'p_cnt', 'putouts', 'assists', 'errors']

In [5]:
rename_home = dict(zip(home_columns, team_columns))
rename_away = dict(zip(away_columns, team_columns))

home_df = data[home_columns]
home_df = home_df.rename(columns=rename_home)

away_df = data[away_columns]
away_df = away_df.rename(columns=rename_away)

away_df.head()
team_df = pd.concat([home_df, away_df], ignore_index=True)
team_df.head()


,team,game_number,AB,H,2B,3B,HR,SF,HBP,BB,SO,p_cnt,putouts,assists,errors
0,ARI,1,26,3,0,0,1,0,1,7,6,6,27,9,0
1,ATL,1,31,4,0,0,1,0,0,5,13,4,27,7,0
2,CHN,1,29,8,3,0,1,1,1,4,7,6,27,9,0
3,SLN,1,31,8,2,0,3,2,1,7,5,4,27,11,0
4,WAS,1,32,6,1,0,1,0,0,2,10,6,27,10,0


In [6]:
team_df = team_df.sort_values(['team','game_number'])

cum_cols = ['AB','H','2B','3B','HR','SF','HBP','BB', 'putouts', 'assists', 'errors']
team_df[cum_cols] = team_df.groupby('team')[cum_cols].cumsum().shift(fill_value=0)

team_df['OBP'] = (team_df['H'] + team_df['BB'] + team_df['HBP']) / (team_df['AB'] + team_df['BB'] + team_df['HBP'] + team_df['SF'])
team_df['SLG'] = (team_df['H'] + 2*team_df['2B'] + 3*team_df['3B'] + 4*team_df['HR']) / team_df['AB']
team_df['OPS'] = team_df['OBP'] + team_df['SLG']

team_df['FPCT'] = (team_df['putouts'] + team_df['assists']) / (team_df['putouts'] + team_df['assists'] + team_df['errors'])

In [47]:
team_metrics_prev = team_df.copy()
team_metrics_prev['game_number'] = team_metrics_prev.groupby('team')['game_number'].shift(-1)
team_metrics_prev.head()

,team,game_number,AB,H,2B,3B,HR,SF,HBP,BB,SO,p_cnt,putouts,assists,errors,OBP,SLG,OPS,FPCT
5,ANA,2.0,0,0,0,0,0,0,0,0,7,6,0,0,0,NaN,NaN,NaN,NaN
13,ANA,3.0,29,4,0,1,0,0,1,1,7,5,27,10,0,0.193548,0.241379,0.434928,1.000000
27,ANA,4.0,69,17,3,1,0,0,1,1,11,4,54,19,0,0.267606,0.376812,0.644417,1.000000
42,ANA,5.0,96,23,4,1,2,0,1,6,7,5,81,33,1,0.291262,0.437500,0.728762,0.991304
52,ANA,6.0,129,30,6,1,3,0,1,7,7,4,108,41,2,0.277372,0.441860,0.719233,0.986755


In [29]:
home_pitcher_columns = ['home_P_id', 'home_game_number', 'game_length', 'home_HR', 'home_HBP', 'home_BB', 'home_SO', 'home_IP']
away_pitcher_columns = ['away_P_id', 'away_game_number', 'game_length', 'away_HR', 'away_HBP', 'away_BB', 'away_SO', 'away_IP']

pitch_columns = ['P_id', 'game_number', 'game_length', 'HR', 'HBP', 'BB', 'SO', 'IP']

In [30]:
rename_homep = dict(zip(home_pitcher_columns, pitch_columns))
rename_awayp = dict(zip(away_pitcher_columns, pitch_columns))

homep_df = data[[c for c in home_pitcher_columns if c != 'home_IP']].copy()
homep_df['home_IP'] = homep_df['game_length'].apply(lambda x: math.floor(x / 6))
homep_df = homep_df.rename(columns=rename_homep)

awayp_df = data[[c for c in away_pitcher_columns if c!= 'away_IP']].copy()
awayp_df['away_IP'] = homep_df['game_length'].apply(lambda x: math.ceil(x / 6))
awayp_df = awayp_df.rename(columns=rename_awayp)

pitcher_df = pd.concat([homep_df, awayp_df], ignore_index=True)
pitcher_df.head()

,P_id,game_number,game_length,HR,HBP,BB,SO,IP
0,bumgm001,1,51,1,1,7,6,8
1,friem001,1,54,1,0,5,13,9
2,hendk001,1,51,1,1,4,7,8
3,waina001,1,51,3,1,7,5,8
4,corbp001,1,54,1,0,2,10,9


In [31]:
pitcher_df = pitcher_df.sort_values(['P_id','game_number'])

cum_cols = ['HR','HBP', 'BB', 'SO', 'IP']
pitcher_df[cum_cols] = pitcher_df.groupby('P_id')[cum_cols].cumsum().shift(fill_value=0)

fip_constant = 3.1
pitcher_df['FIP'] = (13*pitcher_df['HR'] + 3*(pitcher_df['BB'] + pitcher_df['HBP']) - 2*pitcher_df['SO']) / pitcher_df['IP'] + fip_constant

In [48]:
pitcher_metrics_prev = pitcher_df.copy()
pitcher_metrics_prev['game_number'] = pitcher_metrics_prev.groupby('P_id')['game_number'].shift(-1)
pitcher_metrics_prev.head()

,P_id,game_number,game_length,HR,HBP,BB,SO,IP,FIP
1548,abboc001,110.0,51,0,0,0,0,0,NaN
4050,abboc001,114.0,51,3,1,1,11,8,5.975000
1680,abboc001,119.0,54,3,1,4,19,17,4.041176
1756,abboc001,137.0,54,4,2,10,25,26,4.561538
4467,abboc001,142.0,53,4,3,10,31,35,3.928571


In [59]:
df_merged = data[data_columns].merge(team_metrics_prev.add_prefix('home_'), on=['home_team', 'home_game_number'], how='left')

In [60]:
df_merged = df_merged.merge(team_metrics_prev.add_prefix('away_'), on=['away_team', 'away_game_number'], how='left')

In [61]:
df_merged = df_merged.merge(pitcher_metrics_prev.add_prefix('home_'), on=['home_P_id', 'home_game_number'], how='left')

In [62]:
df_merged = df_merged.merge(pitcher_metrics_prev.add_prefix('away_'), on=['away_P_id', 'away_game_number'], how='left')

In [63]:
df_merged.head()

,date,dayofweek,away_team,away_game_number,away_league,home_team,home_game_number,home_league,home_score,away_score,...,home_SO_y,home_IP,home_FIP,away_game_length,away_HR_y,away_HBP_y,away_BB_y,away_SO_y,away_IP,away_FIP
0,20220407,Thu,SDN,1,NL,ARI,1,NL,4,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20220407,Thu,CIN,1,NL,ATL,1,NL,3,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20220407,Thu,MIL,1,NL,CHN,1,NL,5,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20220407,Thu,PIT,1,NL,SLN,1,NL,9,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20220407,Thu,NYN,1,NL,WAS,1,NL,1,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
df_merged['home_won'] = (df_merged['home_score']>df_merged['away_score']).astype(int)
df_final = df_merged[data_columns+['home_OPS', 'home_FIP', 'home_FPCT', 'away_OPS', 'away_FIP', 'away_FPCT', 'home_won']]

In [65]:
df_final.head()

,date,dayofweek,away_team,away_game_number,away_league,home_team,home_game_number,home_league,home_score,away_score,park_id,away_P_id,home_P_id,home_OPS,home_FIP,home_FPCT,away_OPS,away_FIP,away_FPCT,home_won
0,20220407,Thu,SDN,1,NL,ARI,1,NL,4,2,PHO01,darvy001,bumgm001,NaN,NaN,NaN,NaN,NaN,NaN,1
1,20220407,Thu,CIN,1,NL,ATL,1,NL,3,6,ATL03,mahlt001,friem001,NaN,NaN,NaN,NaN,NaN,NaN,0
2,20220407,Thu,MIL,1,NL,CHN,1,NL,5,4,CHI11,burnc002,hendk001,NaN,NaN,NaN,NaN,NaN,NaN,1
3,20220407,Thu,PIT,1,NL,SLN,1,NL,9,0,STL10,brubj001,waina001,NaN,NaN,NaN,NaN,NaN,NaN,1
4,20220407,Thu,NYN,1,NL,WAS,1,NL,1,5,WAS11,megit002,corbp001,NaN,NaN,NaN,NaN,NaN,NaN,0


In [58]:
# metric_blend = (game_number * metrics_current + tau * previous) / (game_number + tau)


,date,dayofweek,away_team,away_game_number,away_league,home_team,home_game_number,home_league,home_score,away_score,park_id,away_P_id,home_P_id,home_OPS,home_FIP,home_FPCT,away_OPS,away_FIP,away_FPCT,home_won
0,20220412,Tue,CLE,5,AL,CIN,5,NL,5,10,CIN09,biebs001,mahlt001,0.515134,4.223656,0.978723,0.868772,4.827599,0.979592,0
4,20220413,Wed,SDN,7,NL,SFN,6,NL,2,1,SFO03,manas001,webbl001,0.708990,1.322222,0.973510,0.802635,3.808738,1.000000,1
5,20220413,Wed,MIL,6,NL,BAL,6,AL,2,4,BAL12,burnc002,meanj001,0.676243,3.978788,0.992647,0.714739,4.528571,0.977941,0
6,20220413,Wed,BOS,6,AL,DET,6,AL,7,9,DET05,eovan001,rodre004,0.684931,4.306642,0.992908,0.626702,3.822222,0.985185,0
8,20220413,Wed,OAK,6,AL,TBA,6,AL,2,4,STP01,montf001,mccls003,0.775430,4.600000,0.985401,0.829524,2.544444,0.992593,0


In [66]:
last_idx = team_df.groupby("team")["game_number"].idxmax()
df_last = team_df.loc[last_idx].reset_index(drop=True)

In [68]:
last_idx_p = pitcher_df.groupby("P_id")["game_number"].idxmax()
df_last_p = pitcher_df.loc[last_idx_p].reset_index(drop=True)

In [69]:
df_last_p

,P_id,game_number,game_length,HR,HBP,BB,SO,IP,FIP
0,abboc001,160,51,5,3,17,61,71,3.142254
1,adonj001,85,51,6,2,27,105,116,2.712069
2,akink001,135,54,6,4,29,117,125,2.644000
3,alcas001,157,51,27,12,83,254,273,3.568864
4,alexj001,146,54,12,2,29,83,90,4.022222
...,...,...,...,...,...,...,...,...,...
362,ynoah001,12,51,0,0,3,8,9,2.322222
363,youna002,137,51,2,0,3,20,18,2.822222
364,zerpa001,97,54,2,0,2,8,9,4.877778
365,zeuct001,122,51,2,1,4,21,18,3.044444
